In [1]:
import pandas as pd

Pré-processamento

In [2]:
base = pd.read_csv('autos.csv', encoding='ISO-8859-1')
base = base.drop('dateCrawled', axis=1)
base = base.drop('dateCreated', axis = 1)
base = base.drop('nrOfPictures', axis = 1)
base = base.drop('postalCode', axis = 1)
base = base.drop('lastSeen', axis = 1)

In [3]:
base['name'].value_counts()

name
Ford_Fiesta                                              657
BMW_318i                                                 627
Opel_Corsa                                               622
Volkswagen_Golf_1.4                                      603
BMW_316i                                                 523
                                                        ... 
Audi_A4_Avant_Klima_Gruene_Plakette_TÜV_&AU_NEU_XENON      1
Renault_clio_in_gold_450VB_!!                              1
Fiat_Doblo_1.6_Multijet                                    1
Renault_Laguna_1                                           1
BMW_M135i_vollausgestattet_NP_52.720____Euro               1
Name: count, Length: 233531, dtype: int64

In [4]:
base = base.drop('name', axis = 1)
base['seller'].value_counts()
base = base.drop('seller', axis = 1)
base['offerType'].value_counts()
base = base.drop('offerType', axis = 1)
i1 = base.loc[base.price <= 10]
base = base[base.price > 10]
i2 = base.loc[base.price > 350000]
base = base[base.price < 350000]

In [5]:
base.vehicleType.value_counts() # limousine
base.gearbox.value_counts() # manuell
base.model.value_counts() # golf
base.fuelType.value_counts() # benzin
base.notRepairedDamage.value_counts() # nein

notRepairedDamage
nein    259301
ja       34004
Name: count, dtype: int64

In [6]:
values = {
    'vehicleType': 'limousine',
    'model': 'golf',
    'gearbox': 'manuell',
    'fuelType': 'benzin',
    'notRepairedDamage': 'nein'
}

In [7]:
base = base.fillna(value = values)

In [8]:
predictors = base.iloc[:, 1:13].values
predictors[0]

array(['test', 'limousine', 1993, 'manuell', 0, 'golf', 150000, 0,
       'benzin', 'volkswagen', 'nein'], dtype=object)

In [9]:
real_price = base.iloc[:, 0].values
real_price

array([  480, 18300,  9800, ...,  9200,  3400, 28990])

In [10]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([
    ("c0", OneHotEncoder(), [0]),
    ("c1", OneHotEncoder(), [1]),
    ("c3", OneHotEncoder(), [3]),
    ("c5", OneHotEncoder(), [5]),
    ("c8", OneHotEncoder(), [8]),
    ("c9", OneHotEncoder(), [9]),
    ("c10", OneHotEncoder(), [10]),
    
    ], remainder='passthrough')

In [11]:
predictors = ct.fit_transform(predictors).toarray()

In [12]:
predictors[:, 3]

array([0., 0., 0., ..., 1., 0., 0.])

In [13]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import cross_val_score
from scikeras.wrappers import KerasRegressor

2023-07-27 17:30:35.953327: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-27 17:30:35.954678: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-27 17:30:35.983342: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-27 17:30:35.983724: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-27 17:30:36.554334: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [14]:
def create_network():
    regressor = Sequential()
    regressor.add(Dense(units=158, activation='relu', input_dim = 316))
    regressor.add(Dense(units=158, activation='relu'))
    regressor.add(Dense(units=1, activation='linear'))

    regressor.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
    regressor.fit(predictors, real_price, batch_size=300, epochs=100)
    return regressor

In [ ]:
regressor = KerasRegressor(build_fn=create_network, epochs=100, batch_size=300)
results = cross_val_score(estimator=regressor, X=predictors, y=real_price, cv =10, scoring='neg_mean_absolute_error')
mean = results.mean()
deviation = results.std()

In [ ]:
# Não rodei por motivos de demorar muito e exigir muito do notebook